<a href="https://colab.research.google.com/github/yardsale8/probability_simulations_in_R/blob/main/2_1_simulating_the_binomial_distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
library(dplyr)
library(tidyr)
library(purrr)
library(devtools)
install_github('yardsale8/purrrfect', force = TRUE)
library(purrrfect)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: usethis





── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp1S9ghI/remotes851c7f1973/yardsale8-purrrfect-d91fae7/DESCRIPTION’ ... OK
* preparing ‘purrrfect’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘purrrfect_1.0.1.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘purrrfect’


The following objects are masked from ‘package:base’:

    replicate, tabulate




# Simulating the Binomial Distribution

In this notebook, we will apply what we've learned to problems related to the binomial distribution.  We will do this by

1. Simulating the raw outcomes using `sample` and converting to the number of success, as well as
2. Simulating the number of successes directly using `rbinom`.

Finally, we will illustrate how to use simulations to estimate the expected value.

### Review - Bernoulli Process

A simple outcomes is generated by a [Bernoulli process](https://en.wikipedia.org/wiki/Bernoulli_process), provided
1. Outcomes are independent,
2. There are two possible outcomes (denoted success and failure), and
3. The probability of a success is contant.

### The binomial distribution

Suppose that we are generating $n$ outcomes from a Bernoulli process with the probability of success given by $p$.  If $X$ is the number of successes in the $n$ trials, then $X$ will have a [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution)

## Strategies for simulating the binomial distribution

1. Simulate the raw Bernoulli trials, then transform into the number of success using either reshaping or `mutate` + `map`.
2. Simulate the number of successes directly using `rbinom`.

### Example - LeBron shoots free throws

LeBron James is NBA player with a stored career.  Over the course of his career, he has made 73.5% of all free throw attempts.  Suppose that we want to model the number of shots he makes in 10 attempts using the binomial distribution.

We will illustrate this process in three ways,

1. Simulating individual shots and reshaping to compute number of successes,
2. Simulating individual shots and using `mutate` + `map` to compute number of successes, and
3. Simulating the number of successing directly using `rbinom`

#### Setting up a sample space

We need to sample from a space with a 73.5% chance of a made free throw, which will be accomplished using a probability vector

In [ ]:
shot <- c('Make', 'Miss')
shot.probs <- c(0.735, 1 - 0.735)

In [ ]:
replicate(10, sample(shot, 10, replace = TRUE, prob = shot.probs))

.trial,.outcome
<dbl>,<list>
1,"Make, Make, Make, Make, Make, Make, Make, Make, Miss, Make"
2,"Make, Make, Make, Make, Make, Miss, Miss, Make, Make, Miss"
3,"Make, Make, Miss, Make, Make, Miss, Make, Miss, Miss, Make"
4,"Make, Make, Miss, Make, Make, Make, Make, Make, Miss, Miss"
5,"Make, Make, Miss, Miss, Make, Make, Make, Miss, Make, Make"
6,"Make, Make, Make, Make, Miss, Make, Miss, Make, Make, Make"
7,"Miss, Make, Make, Miss, Make, Miss, Make, Make, Make, Miss"
8,"Make, Miss, Make, Make, Miss, Make, Miss, Make, Make, Make"
9,"Make, Make, Miss, Make, Make, Make, Make, Make, Make, Make"


#### Approach 1 - Simulate individual shots, reshape, and compute successes.

Comment out lines to explore the output in each step.

In [3]:
N <- 10
(replicate(N, sample(shot, 10, replace = TRUE, prob = shot.probs), .reshape = 'stack')
 %>% mutate(is.success = ifelse(.outcome == 'Make', 1, 0))
 %>% group_by(.trial) %>% summarise(num.successes = sum(is.success))
 )

.trial,num.successes
<dbl>,<dbl>
1,6
2,6
3,7
4,9
5,9
6,7
7,5
8,5
9,6


#### Approach 2 - Simulate individual shots, recode outcomes, and compute successes using `mutate` and `map`

Comment out lines to explore the output in each step.

In [ ]:
N <- 10
(replicate(N, sample(shot, 10, replace = TRUE, prob = shot.probs))
 %>% mutate(is.success = map(.outcome, \(x) ifelse(x == 'Make', 1, 0)))
 %>% mutate(num.successes = map_int(is.success, sum))
 )

.trial,.outcome,is.success,num.successes
<dbl>,<list>,<list>,<int>
1,"Miss, Make, Make, Make, Miss, Make, Miss, Miss, Make, Make","0, 1, 1, 1, 0, 1, 0, 0, 1, 1",6
2,"Make, Make, Make, Miss, Miss, Make, Make, Make, Make, Make","1, 1, 1, 0, 0, 1, 1, 1, 1, 1",8
3,"Make, Make, Miss, Make, Miss, Make, Make, Make, Make, Make","1, 1, 0, 1, 0, 1, 1, 1, 1, 1",8
4,"Miss, Make, Make, Make, Make, Make, Make, Make, Miss, Make","0, 1, 1, 1, 1, 1, 1, 1, 0, 1",8
5,"Make, Make, Make, Make, Make, Make, Make, Make, Make, Miss","1, 1, 1, 1, 1, 1, 1, 1, 1, 0",9
6,"Miss, Make, Miss, Make, Make, Miss, Miss, Make, Make, Make","0, 1, 0, 1, 1, 0, 0, 1, 1, 1",6
7,"Miss, Make, Miss, Miss, Make, Miss, Make, Make, Make, Make","0, 1, 0, 0, 1, 0, 1, 1, 1, 1",6
8,"Make, Make, Miss, Make, Miss, Miss, Make, Make, Miss, Miss","1, 1, 0, 1, 0, 0, 1, 1, 0, 0",5
9,"Miss, Make, Make, Make, Make, Make, Make, Make, Make, Make","0, 1, 1, 1, 1, 1, 1, 1, 1, 1",9


#### Approach 3 - Simulate the number of succeses directly using `rbinom`

Note that the signature of `rbinom` is `rbinom(n, size, prob)` (use help!).  Here will use `n = 1` to mean one experiment per row, `size = 10` to represent the attempts per trial/experiment, and `prob = 0.735`

**Important.** Be sure to use `replicate_int` to get a simple integer `.outcome` column.

In [ ]:
N <- 10
(replicate_int(N, rbinom(1, 10, 0.735))
)

.trial,.outcome
<dbl>,<int>
1,6
2,7
3,10
4,7
5,8
6,5
7,6
8,8
9,5


## Estimating Probabilities

Once we have the column of containing the number of success, we can answer probability questions per usual, e.g., make a Boolean column then use `estimate_prob` or `estimate_all_prob`.

**Example Question.** What the the probability that LeBron will make 9 or more of the 10 free throw attempts?

#### Approach 1 - Stacking the outcomes.

In [13]:
N <- 100000
n <- 10
shot <- c('Make', 'Miss')
shot.probs <- c(0.735, 1 - 0.735)
(replicate(N, sample(shot, 10, replace=TRUE, prob = shot.probs), .reshape = 'stack')
 %>% mutate(is.make = ifelse(.outcome == 'Make', 1, 0))
 %>% group_by(.trial)
 %>% summarise(num.makes = sum(is.make))
 %>% mutate(nine.or.more = num.makes >= 9)
 %>% estimate_all_prob
)

nine.or.more
<dbl>
0.20992


#### Approach 2 -   `mutate` + `map`

In [16]:
N <- 100000
n <- 10
shot <- c('Make', 'Miss')
shot.probs <- c(0.735, 1 - 0.735)
(replicate(N, sample(shot, 10, replace=TRUE, prob = shot.probs))
 %>% mutate(is.make = map(.outcome, \(x) ifelse(x == 'Make', 1, 0)))
 %>% mutate(num.makes = map(is.make, sum))
 %>% mutate(nine.or.more = num.makes >= 9)
 %>% estimate_all_prob
)

nine.or.more
<dbl>
0.21109


#### Approach 3 - Simulate binomial directly with `rbinom`

In [20]:
N <- 100000
(replicate_int(N, rbinom(1, 10, 0.735))
 %>% mutate(nine.or.more = .outcome >= 9)
 %>% estimate_all_prob
)


nine.or.more
<dbl>
0.2112


## Estimating the Distribution

Now that we know how to simulate a binomial random variable, we can use `tabulate` to estimate the values of the probability mass function.

In [ ]:
n <- 10
p <- 0.735
N <- 100000
(replicate_int(N, rbinom(1, n, p))
 %>% tabulate(.outcome)
)

X = 1,X = 10,X = 2,X = 3,X = 4,X = 5,X = 6,X = 7,X = 8,X = 9
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2e-05,0.04508,0.00073,0.00383,0.02098,0.07111,0.16517,0.26106,0.26843,0.16359


## Simulating the Expected Value

So far, we have used our simulating to estimate the answers to probability questions/distributions. Simulations can also be used to simulate the various expected values.  Here are three common types of expectation problems and the associated simulation strategy.

1. **Mean/Expected Value.** Simulate $X$ $\longrightarrow$ summarise using `mean`.
2. **Variance/Standard Deviation.** Simulate $X$ $\longrightarrow$ summarise using `var` or `sd` functions.
3. **Expected Value of a function.** Simulate $X$ $\longrightarrow$ use `mutate` to compute an `f(X)` column $\longrightarrow$ summarise using `mean`.

#### Example 1 - LeBron's mean, variance, and standard deviation number of shots

Again, we have LeBron--a 73.5% free throw shooter--shooting 10 free throws.  Let's estimate the mean, variance, and standard deviation of the number of made shots.  I will use `rbinom` to directly simulate the binomial trials.


In [ ]:
n <- 10
p <- 0.735
N <- 10000
( replicate_int(N, rbinom(1, n, p))
  %>% summarise(approx.mu = mean(.outcome),
                approx.var = var(.outcome),
                approx.sd = sd(.outcome))

)

approx.mu,approx.var,approx.sd
<dbl>,<dbl>,<dbl>
7.3398,1.991135,1.411076


#### Comparing to the theortical values

Next, we can use `mutate` to compute the actual theoretical values and use `relocate` to put the respective values next to each other.

In [ ]:
n <- 10
p <- 0.735
N <- 100000
( replicate_int(N, rbinom(1, 10, 0.735))
  %>% summarise(approx.mu = mean(.outcome),
                approx.var = var(.outcome),
                approx.sd = sd(.outcome))
  %>% mutate(actual.mu = n*p,
             actual.var = n*p*(1-p),
             actual.sd = sqrt(actual.var))
  %>% relocate(actual.mu, .after = approx.mu)
  %>% relocate(actual.var, .after = approx.var)
  %>% relocate(actual.sd, .after = approx.sd)
)



approx.mu,actual.mu,approx.var,actual.var,approx.sd,actual.sd
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7.35553,7.35,1.946488,1.94775,1.395166,1.395618


### <font color="red"> Exercise 2.1.1 - Binomial probilities </font>

Historically circuit boards used in the manufacture of video game consoles are defective 5% of the time. Let $X$ represent the number of defective boards in a random sample of size $n=25$.  

1. Explain why we should use the binomial distribution here.
2. Find $P(X≥5)$ using each of the three approaches discussed above.
3. Use  `tabulate` to estimate the PMF of $X$.
4. Use a simulation to estimate $E(X)$, and $SD(X)$.

